In [1]:
import os,sys
from array import array
import numpy as np
import matplotlib.pyplot as plot
from PIL import Image
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dropout, Dense, Activation,Merge
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
base_dirs = ['caterpillar','case','kubota','doosan','komatsu','sany','volvo','xcmg']

In [3]:
WIDTH = 100
HEIGHT = 100
classes = 8    # 看你是几分类
root = "D://wajueji//"   #目录

In [4]:
#定义图像读取
def  get_image_pixel(file):
    img = Image.open(file)
    img = img.resize((WIDTH,HEIGHT))
    #图片灰度化
    img = img.convert("L")
    img_array = img_to_array(img)
    return img_array

In [6]:
#获取所有图像文件
def get_image_file(root,base_dirs):
    for i in range(len(base_dirs)):
        dir = root + base_dirs[i]
        listImages= os.listdir(dir)
        images_file_path = [ dir+ '//' +image for image in listImages]
        #print(images_file_path)
        for file in images_file_path:
            img_array = get_image_pixel(file)  #这里的i为类别标签
            X.append(img_array)
            Y.append(i)  
    return  X,Y

In [7]:
X = []   #存放训练数据的 X_train 
Y = []   #放训练数据的 Y_train
X,Y = get_image_file(root,base_dirs)

In [8]:
X[0].shape

(100, 100, 1)

In [9]:
total_input = len(X)
X_train = np.array(X)
X_train = X_train.reshape(total_input, WIDTH, HEIGHT, 1)
X_train = X_train.astype('float32')     
X_train /= 255   #归一化
Y_train = np.array(Y)   
Y_train = Y_train.reshape(total_input, 1)   
Y_train = np_utils.to_categorical(Y_train, classes)   #类别编码

In [10]:
X_train.shape

(1545, 100, 100, 1)

In [11]:
len(X_train)

1545

In [14]:
input_size = WIDTH * HEIGHT 
batch_size = 128
hidden_neurons = 30    
epochs = 10

In [15]:

# 构建一个较为简单的CNN模型
model = Sequential()
model.add(Convolution2D(32, (2, 2), input_shape=(WIDTH, HEIGHT, 1)))
model.add(Activation('relu'))
model.add(Convolution2D(32, (2, 2)))  
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))                
model.add(Flatten()) 
model.add(Dense(hidden_neurons))
model.add(Activation('relu'))      
model.add(Dense(classes))
model.add(Activation('softmax'))


In [16]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adadelta')
# optimizer - "rmsprop"/"sgd"/"adadelta" , loss - "binary_crossentropy" / "categorical_crossentropy"

In [18]:
model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_split = 0.2, verbose=1)

Train on 1236 samples, validate on 309 samples
Epoch 1/10
1236/1236 [==============================] - 31s 25ms/step - loss: 3.0885 - acc: 0.2686 - val_loss: 2.8775 - val_acc: 0.0000e+00
Epoch 2/10
1236/1236 [==============================] - 32s 26ms/step - loss: 1.6675 - acc: 0.2791 - val_loss: 3.7666 - val_acc: 0.0000e+00
Epoch 3/10
1236/1236 [==============================] - 31s 25ms/step - loss: 1.5132 - acc: 0.3285 - val_loss: 3.8394 - val_acc: 0.2233
Epoch 4/10
1236/1236 [==============================] - 34s 27ms/step - loss: 1.1181 - acc: 0.5477 - val_loss: 5.2822 - val_acc: 0.2233
Epoch 5/10
1236/1236 [==============================] - 33s 27ms/step - loss: 0.9452 - acc: 0.6650 - val_loss: 6.9296 - val_acc: 0.1974
Epoch 6/10
1236/1236 [==============================] - 32s 26ms/step - loss: 0.4531 - acc: 0.8981 - val_loss: 9.4274 - val_acc: 0.1974
Epoch 7/10
1236/1236 [==============================] - 35s 29ms/step - loss: 0.1815 - acc: 0.9741 - val_loss: 10.9414 - val_acc:

In [19]:
from keras.models import load_model  
model.save('my_model.h5')  
del model 


In [ ]:
from sklearn.externals import joblib
from flask import Flask,request
app = Flask(__name__)

#获取要预测的图片的像素矩阵
def get_test_image_pixel(file):
    X_test = []
    img_array = get_image_pixel(file)
    X_test.append(img_array)
    X_test = np.array(X_test)
    X_test = X_test.reshape(1, WIDTH, HEIGHT, 1)
    X_test = X_test.astype('float32')     
    X_test /= 255   #归一化
    return X_test

#获取最大预测结果的下标
def get_max_index(y):
    y_list = y.tolist()[0]
    index = y_list.index(max(y_list))
    return index

#

def get_label(index):
    return labels[index]

@app.route('/')
def hello_world():
    return 'Hello World!'

@app.route('/predict/<image>')
def predict(image):
    image = 'D://wajueji//case//50.png'
    index = get_max_index(model.predict(get_test_image_pixel(image)))
    return get_label(index)

if __name__ == '__main__':
    #根据base_dirs定义类别列表
    #base_dirs = ['caterpillar','case','kubota','doosan','komatsu','sany','volvo','xcmg']
    labels = ['卡特彼勒','凯斯','久保田','斗山','小松','三一重工','沃尔沃','徐工']
    model = load_model('my_model.h5')
    app.run(host='192.168.31.19')

 * Running on http://192.168.31.19:5000/ (Press CTRL+C to quit)
192.168.31.19 - - [07/Aug/2018 21:42:22] "GET /predict/11 HTTP/1.1" 200 -


In [28]:
#获取要预测的图片的像素矩阵
def get_test_image_pixel(file):
    X_test = []
    img_array = get_image_pixel(file)
    X_test.append(img_array)
    X_test = np.array(X_test)
    X_test = X_test.reshape(1, WIDTH, HEIGHT, 1)
    X_test = X_test.astype('float32')     
    X_test /= 255   #归一化
    return X_test

In [29]:
model = load_model('my_model.h5')

In [30]:
#获取最大预测结果的下标
def get_max_index(y):
    y_list = y.tolist()[0]
    index = y_list.index(max(y_list))
    return index

In [31]:
#根据base_dirs定义类别列表
#base_dirs = ['caterpillar','case','kubota','doosan','komatsu','sany','volvo','xcmg']
labels = ['卡特彼勒','凯斯','久保田','斗山','小松','三一重工','沃尔沃','徐工']
def get_label(index):
    return labels[index]

In [32]:
file = 'D://wajueji//case//50.png'
index = get_max_index(model.predict(get_test_image_pixel(file)))

In [33]:
get_label(index)

'凯斯'